# Standard instances

In [ ]:
import random
import sys

TESTS = 1      # nombre d'instances à générer
MSIZE = 400     # nombre maximum d'objets

def maketest(n, r, pct):
    """
    Génère une instance du problème du sac à dos quadratique.

    Paramètres :
      n   : nombre d'objets
      r   : plage des coefficients (profits et poids seront dans [1, r])
      pct : densité de l'instance (% de profits non nuls)

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) utilisée comme matrice Q
        - w : vecteur des poids (taille n)
        - c : capacité du sac
    """
    # Initialisation de la matrice des profits (Q) et du vecteur des poids
    p = [[0] * n for _ in range(n)]
    w = [0] * n

    # Remplissage de la matrice symétrique
    for i in range(n):
        for j in range(i + 1):
            # Tirage aléatoire entre 0 et 99 pour déterminer si le coefficient est nul
            if random.randrange(100) >= pct:
                val = 0
            else:
                val = random.randrange(r) + 1  # valeur dans [1, r]
            p[i][j] = val
            p[j][i] = val

    # Génération aléatoire des poids : poids dans [1, r//2]
    for i in range(n):
        w[i] = random.randrange(max(r // 2, 1)) + 1

    # Calcul de la somme totale des poids
    wsum = sum(w)
    if wsum - 50 <= 0:
        raise Exception("Somme des poids trop faible pour générer la capacité.")
    
    # Capacité c dans [50, wsum-1]
    c = random.randrange(wsum - 50) + 50

    return {
        'n': n,
        'p': p,
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre,
    au format attendu par la fonction lire_fichier_qkp.

    Format :
      - Ligne 1 : Référence de l'instance
      - Ligne 2 : Nombre de variables (n)
      - Ligne 3 : Coefficients linéaires (diagonale de Q)
      - Lignes 4 à n+2 : n-1 lignes de coefficients quadratiques (partie supérieure de Q)
      - Ligne vide
      - Ligne suivante : Type de contrainte (par exemple 1)
      - Ligne suivante : Capacité C
      - Dernière ligne : Poids des éléments
    """
    n = instance['n']
    Q = instance['p']
    weights = instance['w']
    c = instance['c']
    
    # Référence de l'instance
    file.write(reference + "\n")
    # Nombre de variables
    file.write(f"{n}\n")
    # Coefficients linéaires (diagonale de Q)
    coeff_lin = " ".join(str(Q[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    # Coefficients quadratiques (pour i de 0 à n-2, coefficients Q[i,j] pour j=i+1...n-1)
    for i in range(n - 1):
        coeffs = " ".join(str(Q[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    # Ligne vide
    file.write("\n")
    # Type de contrainte (par exemple 1)
    file.write("1\n")
    # Capacité
    file.write(f"{c}\n")
    # Poids des éléments
    file.write(" ".join(str(w) for w in weights) + "\n")

def main():
    # Lecture des paramètres depuis la ligne de commande ou via input
    if len(sys.argv) == 4:
        try:
            n = int(sys.argv[1])
            r = int(sys.argv[2])
            pct = int(sys.argv[3])
        except ValueError:
            print("Les arguments doivent être des entiers.")
            sys.exit(1)
    else:
        n = int(input("n = "))
        r = int(input("r = "))
        pct = int(input("pct = "))

    if n > MSIZE:
        print("Erreur : n dépasse la taille maximale autorisée ({})".format(MSIZE))
        sys.exit(1)
    
    # Génération de TESTS instances, chacune enregistrée dans un fichier séparé
    for test in range(1, TESTS + 1):
        # Initialisation de la graine pour obtenir des résultats reproductibles
        random.seed(test + n + r + pct)
        try:
            instance = maketest(n, r, pct)
        except Exception as e:
            print("Erreur lors de la génération de l'instance:", e)
            sys.exit(1)
        
        filename = f"instance_qkp_{test}.txt"
        with open(filename, "w") as out_file:
            reference = f"TestInstance{test}"
            print_instance(instance, out_file, reference)
            print(f"Instance {test} enregistrée dans {filename}")

if __name__ == "__main__":
    main()

# instances of dispersion problem

# 1.GEO

In [27]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type GEO (distances euclidiennes).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'cardinality'
        - q : nombre d'installations à placer
    """
    p = [[0] * n for _ in range(n)]
    
    # Générer des positions aléatoires dans un carré 100x100
    positions = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dx = positions[i][0] - positions[j][0]
            dy = positions[i][1] - positions[j][1]
            dist = math.sqrt(dx**2 + dy**2)
            p[i][j] = p[j][i] = dist

    # Contrainte de cardinalité
    q = random.randint(2, n - 2)
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'cardinality',
        'q': q
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    q = instance['q']
    
    file.write(f"# {reference}\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("0\n")  # 0 pour cardinalité
    file.write(f"{q}\n")

def main():
    try:
        TESTS = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_geo50.txt"
    with open(filename, "w") as out_file:
        for test in range(1, TESTS + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceGEO{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < TESTS:
                out_file.write("\n\n")
        print(f"{TESTS} instances GEO enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 50
10 instances GEO enregistrées dans instances_geo50.txt


# 2. WGEO

In [11]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type WGEO (distances euclidiennes pondérées).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij pondérées
        - constraint_type : 'cardinality'
        - q : nombre d'installations à placer
    """
    p = [[0] * n for _ in range(n)]
    
    # Générer des positions aléatoires dans un carré 100x100
    positions = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    # Générer des poids alpha_i dans [5, 10]
    alphas = [random.uniform(5, 10) for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dx = positions[i][0] - positions[j][0]
            dy = positions[i][1] - positions[j][1]
            dist = math.sqrt(dx**2 + dy**2)
            weighted_dist = dist * alphas[i] * alphas[j]
            p[i][j] = p[j][i] = weighted_dist

    # Contrainte de cardinalité
    q = random.randint(2, n - 2)
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'cardinality',
        'q': q
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    q = instance['q']
    
    file.write(reference + "\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("0\n")  # 0 pour cardinalité
    file.write(f"{q}\n")

def main():
    try:
        # Entrée utilisateur pour n
        if len(sys.argv) == 2:
            n = int(sys.argv[1])
        else:
            n = int(input("n = "))
        
        if n <= 0 or n > MSIZE:
            print(f"Erreur : n doit être un entier positif <= {MSIZE}")
            sys.exit(1)
        
        # Entrée utilisateur pour le nombre d'instances
        nb_instances = int(input("Combien d'instances voulez-vous générer ? "))
        if nb_instances <= 0:
            print("Erreur : le nombre d'instances doit être un entier positif")
            sys.exit(1)
        
        # Nom du fichier basé sur n et nb_instances
        filename = f"instances_wgeo_n{n}_count{nb_instances}.txt"
        
        # Ouverture du fichier unique
        with open(filename, "w") as out_file:
            for test in range(1, nb_instances + 1):
                random.seed(test + n)  # Graine pour reproductibilité
                instance = maketest(n)
                reference = f"TestInstanceWGEO50{test}"
                print_instance(instance, out_file, reference)
                # Ajouter deux lignes vides sauf après la dernière instance
                if test < nb_instances:
                    out_file.write("\n\n")
        
        print(f"{nb_instances} instances enregistrées dans {filename}")
    
    except ValueError:
        print("Erreur : veuillez entrer des nombres entiers valides")
    except Exception as e:
        print(f"Erreur inattendue : {e}")

if __name__ == "__main__":
    main()

n = 50
Combien d'instances voulez-vous générer ? 10
10 instances enregistrées dans instances_wgeo_n50_count10.txt


# 3.EXPO

In [32]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type EXPO (distances exponentielles).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'cardinality'
        - q : nombre d'installations à placer
    """
    p = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            # Distribution exponentielle négative avec moyenne 50
            dist = -50 * math.log(random.random())
            p[i][j] = p[j][i] = dist

    # Contrainte de cardinalité
    q = random.randint(2, n - 2)
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'cardinality',
        'q': q
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    q = instance['q']
    
    file.write(f"# {reference}\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("0\n")  # 0 pour cardinalité
    file.write(f"{q}\n")

def main():
    try:
        TESTS = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_expo50.txt"
    with open(filename, "w") as out_file:
        for test in range(1, TESTS + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceEXPO{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < TESTS:
                out_file.write("\n\n")
        print(f"{TESTS} instances EXPO enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 50
10 instances EXPO enregistrées dans instances_expo50.txt


# 4.RAN

In [39]:
import random
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type RAN (distances uniformes).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'cardinality'
        - q : nombre d'installations à placer
    """
    p = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = random.randint(1, 100)
            p[i][j] = p[j][i] = dist

    # Contrainte de cardinalité
    q = random.randint(2, n - 2)
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'cardinality',
        'q': q
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    q = instance['q']
    
    file.write(f"# {reference}\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("0\n")  # 0 pour cardinalité
    file.write(f"{q}\n")

def main():
    try:
        num_instances = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_ran50.txt"
    with open(filename, "w") as out_file:
        for test in range(1, num_instances + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceRAN{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < num_instances:
                out_file.write("\n\n")
        print(f"{num_instances} instances RAN enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 50
10 instances RAN enregistrées dans instances_ran50.txt


# 5.KP_GEO

In [49]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type KP-GEO (distances euclidiennes, contrainte sac à dos).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'knapsack'
        - w : vecteur des poids
        - c : capacité du sac
    """
    p = [[0] * n for _ in range(n)]
    
    # Générer des positions aléatoires dans un carré 100x100
    positions = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dx = positions[i][0] - positions[j][0]
            dy = positions[i][1] - positions[j][1]
            dist = math.sqrt(dx**2 + dy**2)
            p[i][j] = p[j][i] = dist

    # Contrainte de type sac à dos
    w = [random.randint(1, 100) for _ in range(n)]
    c = math.floor(0.5 * sum(w))
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'knapsack',
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    w = instance['w']
    c = instance['c']
    
    file.write(reference + "\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("1\n")  # 1 pour sac à dos
    file.write(f"{c}\n")
    file.write(" ".join(str(weight) for weight in w) + "\n")

def main():
    try:
        num_instances = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_kp_geo200.txt"
    with open(filename, "w") as out_file:
        for test in range(1, num_instances + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceKPGEO{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < num_instances:
                out_file.write("\n\n")
        print(f"{num_instances} instances KP-GEO enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 200
10 instances KP-GEO enregistrées dans instances_kp_geo200.txt


# 6.KP_WGEO

In [50]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type KP-WGEO (distances euclidiennes pondérées, contrainte sac à dos).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij pondérées
        - constraint_type : 'knapsack'
        - w : vecteur des poids
        - c : capacité du sac
    """
    p = [[0] * n for _ in range(n)]
    
    # Générer des positions aléatoires dans un carré 100x100
    positions = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    # Générer des poids alpha_i dans [5, 10]
    alphas = [random.uniform(5, 10) for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dx = positions[i][0] - positions[j][0]
            dy = positions[i][1] - positions[j][1]
            dist = math.sqrt(dx**2 + dy**2)
            weighted_dist = dist * alphas[i] * alphas[j]
            p[i][j] = p[j][i] = weighted_dist

    # Contrainte de type sac à dos
    w = [random.randint(1, 100) for _ in range(n)]
    c = math.floor(0.5 * sum(w))
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'knapsack',
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    w = instance['w']
    c = instance['c']
    
    file.write(reference + "\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("1\n")  # 1 pour sac à dos
    file.write(f"{c}\n")
    file.write(" ".join(str(weight) for weight in w) + "\n")

def main():
    try:
        num_instances = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_kp_wgeo200.txt"
    with open(filename, "w") as out_file:
        for test in range(1, num_instances + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceKPWGEO{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < num_instances:
                out_file.write("\n\n")
        print(f"{num_instances} instances KP-WGEO enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 200
10 instances KP-WGEO enregistrées dans instances_kp_wgeo200.txt


# 7.KP_EXPO

In [51]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type KP-EXPO (distances exponentielles, contrainte sac à dos).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'knapsack'
        - w : vecteur des poids
        - c : capacité du sac
    """
    p = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            # Distribution exponentielle négative avec moyenne 50
            dist = -50 * math.log(random.random())
            p[i][j] = p[j][i] = dist

    # Contrainte de type sac à dos
    w = [random.randint(1, 100) for _ in range(n)]
    c = math.floor(0.5 * sum(w))
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'knapsack',
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    w = instance['w']
    c = instance['c']
    
    file.write(reference + "\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("1\n")  # 1 pour sac à dos
    file.write(f"{c}\n")
    file.write(" ".join(str(weight) for weight in w) + "\n")

def main():
    try:
        num_instances = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_kp_expo200.txt"
    with open(filename, "w") as out_file:
        for test in range(1, num_instances + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceKPEXPO{test}"
            print_instance(instance, out_file, reference)
            # Ajouter deux lignes vides après chaque instance, sauf après la dernière
            if test < num_instances:
                out_file.write("\n\n")
        print(f"{num_instances} instances KP-EXPO enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 200
10 instances KP-EXPO enregistrées dans instances_kp_expo200.txt


# 8.KP_RAN

In [52]:
import random
import math
import sys

MSIZE = 400  # nombre maximum d'objets

def maketest(n):
    """
    Génère une instance de type KP-RAN (distances uniformes, contrainte sac à dos).

    Paramètres :
      n : nombre d'objets

    Retourne :
      Un dictionnaire contenant :
        - n : nombre d'objets
        - p : matrice symétrique (n x n) des distances d_ij
        - constraint_type : 'knapsack'
        - w : vecteur des poids
        - c : capacité du sac
    """
    p = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = random.randint(1, 100)
            p[i][j] = p[j][i] = dist

    # Contrainte de type sac à dos
    w = [random.randint(1, 100) for _ in range(n)]
    c = math.floor(0.5 * sum(w))
    
    return {
        'n': n,
        'p': p,
        'constraint_type': 'knapsack',
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre.
    """
    n = instance['n']
    p = instance['p']
    w = instance['w']
    c = instance['c']
    
    file.write(reference + "\n")
    file.write(f"{n}\n")
    coeff_lin = " ".join(str(p[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    for i in range(n - 1):
        coeffs = " ".join(str(p[i][j]) for j in range(i+1, n))
        file.write(coeffs + "\n")
    file.write("\n")
    file.write("1\n")  # 1 pour sac à dos
    file.write(f"{c}\n")
    file.write(" ".join(str(weight) for weight in w) + "\n")

def main():
    try:
        num_instances = int(input("Nombre d'instances à générer : "))
        n = int(input("n = "))
    except ValueError:
        print("Les entrées doivent être des entiers.")
        sys.exit(1)

    if n > MSIZE:
        print(f"Erreur : n dépasse la taille maximale autorisée ({MSIZE})")
        sys.exit(1)
    
    filename = "instances_kp_ran200.txt"
    with open(filename, "w") as out_file:
        for test in range(1, num_instances + 1):
            random.seed(test + n)
            instance = maketest(n)
            reference = f"TestInstanceKPRAN{test}"
            print_instance(instance, out_file, reference)
            if test < num_instances:
                out_file.write("\n\n")
        print(f"{num_instances} instances KP-RAN enregistrées dans {filename}")

if __name__ == "__main__":
    main()

Nombre d'instances à générer : 10
n = 200
10 instances KP-RAN enregistrées dans instances_kp_ran200.txt


# Instances of Densest Subgraphs

In [11]:
import random

MSIZE = 400  # nombre maximum de nœuds

def generate_densest_subgraph_instance(n, density_pct):
    """
    Génère une instance du problème du sous-graphe le plus dense formulé comme un QKP.

    Paramètres :
      n          : nombre de nœuds dans le graphe
      density_pct: densité des arêtes en pourcentage (0 à 100)

    Retourne :
      Un dictionnaire contenant :
        - n : nombre de nœuds
        - Q : matrice d'adjacence symétrique (n x n) avec Q[i][i] = 0
        - w : vecteur des poids (tous égaux à 1)
        - c : capacité du sac (égale à q, nombre de nœuds à sélectionner)
    """
    if n < 2:
        raise ValueError("n doit être au moins 2 pour avoir un sous-graphe significatif.")
    
    # Initialisation de la matrice d'adjacence Q
    Q = [[0] * n for _ in range(n)]

    # Probabilité qu'une arête soit présente
    prob = density_pct / 100.0

    # Remplissage de la matrice symétrique Q
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < prob:
                Q[i][j] = 1
                Q[j][i] = 1
            else:
                Q[i][j] = 0
                Q[j][i] = 0

    # Les éléments diagonaux sont 0 (pas de boucles)
    for i in range(n):
        Q[i][i] = 0

    # Poids des nœuds : tous égaux à 1
    w = [1] * n

    # Générer q aléatoirement dans [2, n-2]
    if n <= 3:
        q = 2  # Pour n=2 ou n=3, q doit être 2
    else:
        q = random.randint(2, n - 2)

    # Capacité c = q
    c = q

    return {
        'n': n,
        'Q': Q,
        'w': w,
        'c': c
    }

def print_instance(instance, file, reference):
    """
    Enregistre l'instance dans le fichier passé en paramètre,
    au format attendu par la fonction lire_fichier_qkp.

    Format :
      - Ligne 1 : Référence de l'instance
      - Ligne 2 : Nombre de variables (n)
      - Ligne 3 : Coefficients linéaires (diagonale de Q, ici tous 0)
      - Lignes 4 à n+2 : n-1 lignes de coefficients quadratiques (partie supérieure de Q)
      - Ligne vide
      - Ligne suivante : Type de contrainte (par exemple 1)
      - Ligne suivante : Capacité c
      - Dernière ligne : Poids des éléments (tous 1)
    """
    n = instance['n']
    Q = instance['Q']
    weights = instance['w']
    c = instance['c']
    
    # Référence de l'instance
    file.write(reference + "\n")
    # Nombre de variables
    file.write(f"{n}\n")
    # Coefficients linéaires (diagonale de Q, tous 0)
    coeff_lin = " ".join(str(Q[i][i]) for i in range(n))
    file.write(coeff_lin + "\n")
    # Coefficients quadratiques (pour i de 0 à n-2, coefficients Q[i,j] pour j=i+1...n-1)
    for i in range(n - 1):
        coeffs = " ".join(str(Q[i][j]) for j in range(i + 1, n))
        file.write(coeffs + "\n")
    # Ligne vide
    file.write("\n")
    # Type de contrainte (par exemple 1)
    file.write("1\n")
    # Capacité c
    file.write(f"{c}\n")
    # Poids des éléments (tous 1)
    file.write(" ".join(str(w) for w in weights) + "\n")

def main():
    try:
        # Demander à l'utilisateur d'entrer le nombre de nœuds
        n = int(input("Entrez le nombre de nœuds (n) : "))
        if n <= 0 or n > MSIZE:
            raise ValueError(f"n doit être un entier positif <= {MSIZE}")
        
        # Demander le pourcentage de densité
        density_pct = int(input("Entrez le pourcentage de densité des arêtes (0 à 100) : "))
        if density_pct < 0 or density_pct > 100:
            raise ValueError("Le pourcentage de densité doit être entre 0 et 100")
        
        # Demander le nombre d'instances à générer
        num_instances = int(input("Entrez le nombre d'instances à générer : "))
        if num_instances <= 0:
            raise ValueError("Le nombre d'instances doit être un entier positif")
        
        # Déterminer le type d'instance en fonction du pourcentage
        type_map = {25: "DSUB25", 50: "DSUB50", 75: "DSUB75", 90: "DSUB90"}
        instance_type = type_map.get(density_pct, f"pct{density_pct}")
        
        # Nom du fichier incluant le nombre d'instances
        filename = f"densest_subgraph_{instance_type}_n{n}_instances{num_instances}.txt"
        
        # Ouvrir le fichier une seule fois pour toutes les instances
        with open(filename, "w") as out_file:
            first_instance = True
            for i in range(num_instances):
                # Ajouter deux lignes vides avant chaque instance sauf la première
                if not first_instance:
                    out_file.write("\n\n")
                else:
                    first_instance = False
                
                # Génération de l'instance
                instance = generate_densest_subgraph_instance(n, density_pct)
                
                # Créer une référence unique pour chaque instance
                reference = f"DensestSubgraphInstance_{instance_type}_instance{i+1}"
                
                # Écrire l'instance dans le fichier
                print_instance(instance, out_file, reference)
        
        print(f"{num_instances} instances enregistrées dans {filename}")
    
    except ValueError as ve:
        print(f"Erreur de saisie : {ve}")
    except Exception as e:
        print(f"Erreur lors de la génération des instances : {e}")

if __name__ == "__main__":
    main()

Entrez le nombre de nœuds (n) : 400
Entrez le pourcentage de densité des arêtes (0 à 100) : 75
Entrez le nombre d'instances à générer : 10
10 instances enregistrées dans densest_subgraph_DSUB75_n400_instances10.txt


# Instances of Planted Cliques

In [ ]:
import random
import math
import sys

TESTS = 10  # Nombre d'instances à générer
MSIZE = 400  # Taille maximale de n

def maketest(n, k, clique_nodes):
    """
    Génère une instance avec une clique plantée.
    Paramètres :
      n : nombre de nœuds
      k : taille de la clique
      clique_nodes : liste des nœuds formant la clique
    """
    if k > n:
        raise ValueError("k ne peut pas être supérieur à n.")
    
    # Initialisation de la matrice p
    p = [[0] * n for _ in range(n)]
    
    # Graphe aléatoire Erdös-Rényi (p = 1/2)
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.5:
                p[i][j] = 1
                p[j][i] = 1
    
    # Création de la clique avec les nœuds sélectionnés
    for i in clique_nodes:
        for j in clique_nodes:
            if i != j:
                p[i][j] = 1
                p[j][i] = 1
    
    w = [1] * n  # Poids
    c = k        # Capacité
    
    return {'n': n, 'p': p, 'w': w, 'c': c}

def main():
    n = int(input("n = "))
    if n > MSIZE:
        print(f"Erreur : n dépasse {MSIZE}")
        sys.exit(1)
    
    k = math.floor(math.sqrt(n))  # k fixe basé sur n
    valeurs_optimales = []
    
    with open("instances_qkp.txt", "w") as out_file:
        for test in range(1, TESTS + 1):
            random.seed(test + n + k)  # Graine différente pour chaque instance
            try:
                # Sélection aléatoire des nœuds pour la clique
                clique_nodes = random.sample(range(n), k)
                print(f"Instance {test} : Nœuds de la clique : {clique_nodes}")
                
                instance = maketest(n, k, clique_nodes)
                valeur_optimale = k * (k - 1)
                print(f"Valeur optimale instance {test} : {valeur_optimale}")
                valeurs_optimales.append(valeur_optimale)
            except Exception as e:
                print("Erreur :", e)
                sys.exit(1)
    
    if valeurs_optimales:
        moyenne = sum(valeurs_optimales) / len(valeurs_optimales)
        print(f"Moyenne des valeurs optimales : {moyenne:.2f}")

if __name__ == "__main__":
    main()